# Analyze motion data from SenseHAT to detect figures

With the TjBot based on Raspberry Pi the motion sensor data (acc, gyro, compass, ..) from the SenseHAT will be pushed through the IoT Foundation via MQTT into a cloudant DB. With this python notebook you will be able to collect this data and create some visualisations.

With this python notebook you will be able to 

* collect this data and create some visualisations
* and to train a support vector classifier

## Install the necessary python libraries missing by default

In [ ]:
# install missing library for cloudant
!pip install cloudant

## Get the credentials to access the cloudant DB

Use the existing connection document (CloudandDB) and push "insert code" rename it to credentials.

In [ ]:
# The code was removed by Watson Studio for sharing.

## Import the cloudant client 

In [ ]:
from cloudant.client import Cloudant
from cloudant.result import Result, ResultByKey
from cloudant.query import Query

client = Cloudant(credentials['username'], credentials['password'], 
                  url=credentials['custom_url'], connect=True)

### Select the database

In [ ]:
database  = client['sensehat_motion']

## Construct a Query for Motion objects

maybe query as a sorted list but needs index in the DB: 
    sort=['payload.d.device', 'payload.d.userid','payload.d.figure','payload.d.motionset', 'payload.d.date']

In [ ]:
# Select statement for document selection
# filter on one the date element to collect the subset
selector = {
    '_id':{'$gt': 0},
    'payload.d.motionset': {
        '$gte': '2018-12-16T08:00',  # $gte: greater than or equal
        '$lt': '2018-12-17T00:00'},  # $lte: less than or equal
}

# Selected fields of the document
fields = [
    'payload.d.acceleration',
    'payload.d.gyroscope',
    'payload.d.orientation',
    'payload.d.compass',
    'payload.d.device',    
    'payload.d.userid',
    'payload.d.figure',
    'payload.d.motionset',
    'payload.d.timestamp',
    'payload.d.date']

# Create the query and get a handler
motion_query = Query (
    database,
    selector=selector, 
    fields=fields
)

In [ ]:
# show 5 elements from the cloudantDB
for doc in motion_query(limit=5) ['docs']:
    print(doc)

# Using pandas for the data processing

In [ ]:
import pandas as pd
from pandas import date_range, to_datetime
from pandas.io.json import json_normalize
from pandas import Timestamp, DataFrame, Series, Timedelta, concat

## Store the data in an array as a table

Also rename the columns for better reading afterwards

In [ ]:
# get all json-objects (documents) out of the query 
json_docs = motion_query()['docs']

# normalize into a dataframe
df = json_normalize(json_docs)

df.head()

In [ ]:
# rename all columns into clear names
df = df.rename(
    columns= {
    'payload.d.acceleration.x' : 'acc_x',
    'payload.d.acceleration.y' : 'acc_y',
    'payload.d.acceleration.z' : 'acc_z',
    'payload.d.gyroscope.x' : 'gyro_x',
    'payload.d.gyroscope.y' : 'gyro_y',
    'payload.d.gyroscope.z' : 'gyro_z',
    'payload.d.orientation.roll' : 'roll',
    'payload.d.orientation.pitch' : 'pitch',
    'payload.d.orientation.yaw' : 'yaw',
    'payload.d.compass':'compass',
    'payload.d.device':'device',    
    'payload.d.userid':'userid',
    'payload.d.figure':'figure',
    'payload.d.motionset':'motionset',
    'payload.d.timestamp':'timestamp',
    'payload.d.date':'date'        
    }
)
df.head()

## Reorganize the array and sort

In [ ]:
# get the columns names
cols = df.columns.tolist()
cols

In [ ]:
# reorder the columns from the array
cols = [
    'device',
    'userid',    
    'figure',
    'motionset',
    'date',
    'timestamp',
    'acc_x',
    'acc_y',
    'acc_z',
    'gyro_x',
    'gyro_y',
    'gyro_z',
    'pitch',
    'roll',
    'yaw',
    'compass'
]

df = df[cols]
df.head()

### change the values to its datatype and sort the values 

In [ ]:
df['date'] = to_datetime(df.date)
df['motionset'] = to_datetime(df.motionset)
df['figure'] = [str(l) for l in df.figure]
df = df.set_index('date').sort_index()
df = df.sort_values(['device','userid','figure','motionset'])
df.head()

In [ ]:
# show the end of the dataframe
df.tail()

# Organize all figures out of the training set into an directory

In [ ]:
from collections import defaultdict

## store figures & motions into a directory 

create a loop of all figures and store each motionset into a dataframe (appand into an array on each figure)

In [ ]:
# loop only for 5 motionsets for demonstration only
for (figure, motionset_id), motionset_data in list(df.groupby(['figure', 'motionset']))[:5]:
    print(figure, motionset_id, len(motionset_data), type(motionset_data))

### Usage of special directories

In [ ]:
# classical directories give a failure if the element doesn't exist
d={}
# d['a']

In [ ]:
# special directory which allows asking elements without failure when not exists
d = defaultdict(list)
d['a']

### store all motions into motionset

motionset will be the overall array of every figures

In [ ]:
# using groupby function on the motions dataset
motionset = defaultdict(list)
for (figure, _), data in df.groupby(['figure', 'motionset']):
    motionset[figure].append(data)

# give the amount of each figure and its stored motions
for figure, datasets in sorted(motionset.items()):
    print ("'{}' : {} recorded motions".format(figure, len(datasets)))

# Plot examples to gain insights of the motionsets

In [ ]:
import matplotlib.pyplot as plt

## Sample of one figure

In [ ]:
# first array is the figure, second is one motionset 
motionset['1'][-1][['acc_x', 'acc_y', 'acc_z']].plot(grid=True, figsize=(15,5));

### Sample Plot on each figure

In [ ]:
# get all figures and plot one figure (the last motion for each figure: -1)
for k in motionset.keys():
    motionset[k][-1][['acc_x', 'acc_y', 'acc_z']].plot(title=k, grid=True, figsize=(15,5))

### Sample Plot on each figure multiple motionsets

to get insights into the difference of each motionset on the same figure

In [ ]:
#  print up to 3 on one figure to compare its behavior
# get all the possible figures
for fig in sorted(motionset.keys()):

    # max 2 plots
    l = min(2, len(motionset[fig]))

    # print max l on each figure
    for i in range(l):
        motionset[fig][i][['acc_x', 'acc_y', 'acc_z']].plot(title='Figure:' + fig + ' Num:' + str(i), grid=True, figsize=(5,3))


In [ ]:
# Subplots are organized in a Rows x Cols Grid
figures = sorted(motionset.keys())
Tot = len(figures)
Cols = 2

# Compute Rows required
Rows = Tot // Cols 
Rows += Tot % Cols

print('Total:' + str(Tot) + ' Rows: ' + str(Rows) + ' Cols:' + str(Cols))

# Create a Position index
Position = range(1,Tot + 1)

# Create main figure
fig = plt.figure(1)
fig.set_size_inches(20,25)

# Create a figure on each sample motionset
i = 0 
for k in figures:    
    # add every single subplot to the figure with a for loop
    ax = fig.add_subplot(Rows,Cols,Position[i])
    df = motionset[k][-2]
    ax.plot(df.index, df[['acc_x']], label='Accel-x')
    ax.plot(df.index, df[['acc_y']], label='Accel-y')
    ax.plot(df.index, df[['acc_z']], label='Accel-z')

    ax.set_title('Sample of figure: '+ k)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    ax.grid(True)
    i+=1
    
plt.show()

# Build a training data set

* The idea is to transform all figures into equally long montions.
* Then we can pass them into a classical machine learning algorithms like a support vecotor classifier.
* We will achieve this by interpolating along the time axis.
* We concentrate on the accelleration features first


In [ ]:
features = ['acc_x', 'acc_y', 'acc_z']

In [ ]:
df = motionset['0'][2][features + ['timestamp']]
df = df.set_index('timestamp')
df.index = df.index - df.index.min()

In [ ]:
title="#points={:g}".format(len(df))
df.plot(style=['d--', 'd--', 'd--', ], grid=True, figsize=(15,5), title=title);

#### Make a regular time index from minimum to maximum with $n$ points

* Use numpy's linear interpolation function interp

In [ ]:
import numpy as np

In [ ]:
t1, t2 = df.index.min(), df.index.max()
new_index = np.linspace(t1, t2, 25)

In [ ]:
np.interp(new_index, df.index, df.values[:,1])

In [ ]:
ip = DataFrame(
    data = dict((col,np.interp(new_index, df.index, df[col].values)) for col in df),
    index = new_index
)
ip

In [ ]:
title="original data (n={:g})".format(len(df))
df.plot(style=['d--', 'd--', 'd--', ], grid=True, figsize=(15,3), title=title);

title="interpolated data (n={:g})".format(len(ip))
ip.plot(style=['.-', '.-', '.-', ], grid=True, figsize=(15,3), title=title);

### Combine everything into a function to make a normalized time series for each figure motion

In [ ]:
def make_normalized_data(df, w=25):

    df = df.set_index('timestamp')
    df.index = (df.index - df.index.min())
    
    t1, t2 = df.index.min(), df.index.max()
    
    new_index = np.linspace(t1, t2, w)

    interp = DataFrame(
        data = dict((col, np.interp(new_index, df.index, df[col].values)) for col in df),
        index = new_index
    )
    return interp

## Data cleaning

#### Each recored motion shall have enough accelaration values

In [ ]:
df = motionset['0'][0][features + ['timestamp']]
ip = make_normalized_data(df)
ip.plot(grid=True, figsize=(15,5), title=title);

In [ ]:
ip.var().sum()

#### Each recorded motion shall have a proper duration (i.e. prober number of messages)

In [ ]:
from pandas import Series
import numpy as np

# make a list of observed length
counts = [len(df) for df in motionset['1']]
print(counts)

# compute a robust estimate of the typical length
counts = Series(counts)
q25 = np.floor(counts.quantile(0.25))
q75 = np.ceil(counts.quantile(0.75))
print("quantiles:", q25, q75)

lower = q25 - 2*(q75-q25)
upper = q75 + 2*(q75-q25)
print("bounds:", lower, upper)

#### Padas detour: A Dataframe can easily be reshape into a vector

In [ ]:
ip.values

In [ ]:
ip.values.reshape(-1)

##### Note

The `ip.values` array is row-major. That means, that `ip.values.reshape(-1)` yields a vector where the first elements are the first *row* of `ip.values`, then the following rows are concatenated.

More general. If $A=[a_{ij}]$ for $i = 1, \ldots, m$ and $j=1, \ldots n$ and A is row-major `b = A.rehsape(-1)` will yield the vector $b = [a_{1,1}, \ldots, a_{1,n}, a_{2,1}, \ldots, a_{2,n}, \ldots, a_{m,1}, \ldots, a_{m,n}]$ 

## Build a list of relevant feature vectors and labels out of motion set

In [ ]:
features = ['acc_x' ,'acc_y', 'acc_z'] # + ['gyro_x', 'gyro_y', 'gyro_z']

# vectors will contain all relevant feature vectors
vectors = []

# the corresponfing labels
labels = []

# loop over all motion sets
for figure, datasets in motionset.items():
    
    # comput robus upper and lower bounds on length
    counts = Series([len(df) for df in datasets])
    q25 = np.floor(counts.quantile(0.25))
    q75 = np.ceil(counts.quantile(0.75))
    lower = q25 - 2*(q75-q25)
    upper = q75 + 2*(q75-q25)
    
    # for each data set ...
    for df in datasets:
        # ... check its length
        if lower <= len(df) <= upper:
            # if long enough compute its normalized version
            ip = make_normalized_data(df[features + ['timestamp']])
            
            # if it has sufficient variance add it as a training example
            variance =  ip.var().sum()
            if variance > 0.01:
                vectors.append(ip.values.reshape(-1))
                labels.append(figure)
            else:
                print("Skipping motion for '{}': total variance {} to small.".format(figure, variance))
        else:
            print("Skipping motion for '{}': length {} not in range [{}, {}]".format(figure, len(df), lower, upper))

### Convert to design matrix X and label vector Y

In [ ]:
X = DataFrame(vectors)
Y = Series(labels)

In [ ]:
X.describe()

In [ ]:
Y.describe()

# Finally apply machine learning to build a motion classifier

## Directly apply a support vector classifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = SVC()
svm.fit(X,Y)

### Evaluate what has been learnt

In [ ]:
y_pred = svm.predict(X)

#### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

labels=sorted(motionset.keys())
C = confusion_matrix(Y, y_pred, labels=labels)
C

In [ ]:
# format it as a DataFrame (for nice visual)
C = DataFrame(C, columns=labels, index=labels)
C.index.name='true'
C.columns.name='pred'
C

#### Recall / Precision / F1

* Recall ($r$): Percentage of class which was classified correctly
* Precision ($p$): Percentage of predictions of a class which are predicted correctly
* F1: $2\frac{r \cdot p}{r + p}$ ... $0 \le F1 \le 1$

In [ ]:
from sklearn.metrics.classification import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
accuracy_score(Y, y_pred)

In [ ]:
recall_score(Y, y_pred, average='weighted')

In [ ]:
precision_score(Y, y_pred, average='weighted')

In [ ]:
f1_score(Y, y_pred, average='weighted')

In [ ]:
from sklearn.metrics.classification import classification_report, f1_score, precision_score, recall_score
print(classification_report(Y, y_pred))

##### Detailed look on wrong classification

In [ ]:
wrong = np.where(y_pred != Y)[0]
DataFrame(
    data=[[i, Y[i], y_pred[i]] for i in wrong],
    columns=['Example', 'True', 'Predicted']
)

## Make better data preparation

### Make all features 'similar' (standardization)

* subtract mean
* divide by standard deviation

In [ ]:
plt.plot(X.mean());
plt.grid(True);

In [ ]:
plt.plot(X.std());
plt.grid(True);

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X0 = scaler.transform(X)

In [ ]:
plt.plot(X0.mean(axis=0));
plt.grid(True);

In [ ]:
plt.plot(X0.std(axis=0));
plt.grid(True);

## Combine the scaler and a classifier

In [ ]:
model = Pipeline([
    ('scale', StandardScaler()),
    ('svc', SVC()),
])

In [ ]:
model.fit(X,Y)
y_pred = model.predict(X)

In [ ]:
labels=sorted(motionset.keys())
C = confusion_matrix(Y, y_pred, labels=labels)
C

In [ ]:
y_pred_train = model.predict(X)
print(classification_report(Y, y_pred_train))

### Assess the generalization capability by crossvalidation

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
cv = cross_validate(model, X, Y, cv=10, return_train_score=False)
DataFrame(data=cv)

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
y_pred_cv = cross_val_predict(model, X, Y, cv=10)

In [ ]:
wrong = np.where(y_pred_cv != Y)[0]
DataFrame(
    data=[[i, Y[i], y_pred_cv[i]] for i in wrong],
    columns=['Example', 'True', 'Predicted']
)

In [ ]:
print(classification_report(Y, y_pred_cv))

### Let's try to improve the classifier

In [ ]:
model = Pipeline([
    ('scale', StandardScaler()),
    ('svc', SVC(kernel='rbf')),
])

In [ ]:
y_pred_cv = cross_val_predict(model, X, Y, cv=10)
print(classification_report(Y, y_pred_cv))

### Automatic meta parameter search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
gamma0 = 1.0 / (X.shape[1] * X.std().std())

svm_cv = GridSearchCV(
            estimator = SVC(kernel='rbf'),
            param_grid = [
                # variations with the RBF kernel
                dict(
                    kernel=['rbf'],
                    C=[1, 0.1, 0.01],
                    gamma=np.array([1/10, 1/5, 1/2, 1.0, 2, 4])*gamma0
                ),
                # variations with a linear kernel
                dict(
                    kernel=['linear'],
                    C=[1, 0.1, 0.01]
                )
            ],
            cv = 10,
            iid=True,
            verbose = 1
        )

model = Pipeline([
    ('scale', StandardScaler()),
    ('svc', svm_cv),
])

In [ ]:
model.fit(X,Y)

In [ ]:
model.named_steps['svc'].best_params_ 

In [ ]:
y_pred_cv = cross_val_predict(model, X, Y, cv=10)
print(classification_report(Y, y_pred_cv))

In [ ]:
wrong = np.where(y_pred_cv != Y)[0]
DataFrame(
    data=[[i, Y[i], y_pred_cv[i]] for i in wrong],
    columns=['Example', 'True', 'Predicted']
)

# Train final model and deploy to Watson Machine Learning (WML)

### Prerequisits

First one has to initialize a machine learning model service and ad credetials for this service

### Install and import machine learning client library and copy the credentials in a hidden cell

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
# The code was removed by Watson Studio for sharing.

### Instantiate a client

In [ ]:
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
import json

instance_details = wml_client.service_instance.get_details()
print(json.dumps(instance_details, indent=2))

#### Build final model

In [ ]:
final = model = Pipeline([
    ('scale', StandardScaler()),
    ('svc', SVC(kernel='rbf')),
])

y_pred_cv = cross_val_predict(final, X, Y, cv=10)
print(classification_report(Y, y_pred_cv))

final.fit(X,Y)

In [ ]:
model_props = {
    wml_client.repository.ModelMetaNames.AUTHOR_NAME: "Thomas Natschläger", 
    wml_client.repository.ModelMetaNames.AUTHOR_EMAIL: "thomas.natschlaeger@gmail.com",
    wml_client.repository.ModelMetaNames.NAME: "Motion based digit classification"
}
model_props

#### Publishing the model

In [ ]:
published_model = wml_client.repository.store_model(model=final, meta_props=model_props, training_data=X, training_target=Y)

In [ ]:
wml_client.repository.list_models()

In [ ]:
published_model_uid = wml_client.repository.get_model_uid(published_model)
model_details = wml_client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

#### Creating a deployment (i.e. the callable WEB service)

In [ ]:
created_deployment = wml_client.deployments.create(published_model_uid, "Deployment digit classifier")

In [ ]:
print(json.dumps(created_deployment, indent=2))

# Use the deployed model via the client

In [ ]:
model_scoring_url = wml_client.deployments.get_scoring_url( created_deployment )
model_scoring_url

In [ ]:
wml_client.deployments.score( model_scoring_url, { "values" : [list(X.iloc[1,:])] } )

In [ ]:
Y.iloc[1]

# Create a function which does the preprocessing and calls the deployed model

For more details see https://dataplatform.cloud.ibm.com/docs/content/analyze-data/ml-deploy-functions.html

In [ ]:
parms = { 
    'wml_credentials' : wml_credentials,
    'model_scoring_url' : model_scoring_url,
    'n_interp': 25,
    'min_number_of_events': 6
}

def digit_classification( parms=parms ):
    
    def make_feature_matrix(values):
        """
        values is the array of motion events
        """
        import numpy as np
        # parameter:
        #   w ... number of points for interpolation. Must be the same as during learning the model
        w = parms['n_interp']

        # time stamps of recorded data in form of numpy vector
        t_rec = np.array([e['timestamp'] for e in values])

        # accelaration data in form of a numpy array of shape (len(payload),3)
        a_rec = np.array([
            [e['acceleration'][col] for col in ('x', 'y', 'z')] for e in values
        ])

        # make a regular linear space from the begining to the end
        t_int = np.linspace(t_rec[0], t_rec[-1], w)

        # now interpolate the x, y, and z coordinate
        a_int = np.zeros((w, 3))
        for j in (0,1,2):
            a_int[:,j] = np.interp(t_int, t_rec, a_rec[:,j])

        return a_int
        
    def score(payload):
            
        try:
            
            # we need WML client to be able to call the previously deployed SVC model
            from watson_machine_learning_client import WatsonMachineLearningAPIClient
            client = WatsonMachineLearningAPIClient( parms["wml_credentials"] )

            # values is the array of motion events
            values = payload['values']
            
            # if we have only a very small number of events we return "<to-short>"
            if len(values) < parms['min_number_of_events']:
                 return {"figure" : "<to-short>"}
            
            # convert the JSON data structure into a numpy array
            data = make_feature_matrix(values)
            
            # compute the sum of the variances of the x, y, z acceleration signals
            variance = data.var(axis=0).sum()
            
            # if the variance is to low there was probably no motion and we return "<no-motion>""
            if variance < 0.01:
                return {"figure" : "<no-motion>"}
            
            
            # the depoyed model requires a scoring_payload with a field names 'values'.
            # where the values must be a list of feature vectors. Each feature vector
            # has to have the required dimension (75 in our case) and must be a plain
            # python list (a numpy array does not work bcs it is not JSON serializable).
            scoring_payload = {'values': [list(data.reshape(-1))]}
            
            # now we call the model via the REST API endpoint at the scoring url
            model_result = client.deployments.score( parms["model_scoring_url"], scoring_payload )
            
            # the result is a dictionaory where in the field 'values' the predictions are stored
            # the [0][0] is required as values is a list of predictions (one for each feature vector in
            # the scoring payload) and each prediction is a vector. In our case each such vector only has
            # one entry (the classification) but it may be that there are multiple outputs (e.g. for a neural network)
            digit_class  = model_result["values"][0][0]
            
            # return the class of the digit/motion
            return { "figure" : digit_class }
            
        except Exception as e:
            return { "error" : repr( e ) }

    return score

### Test the function with some data from ClaudantDB

**NOTE: You have to adapt the selector for your database. In particulare the `payload.d.motionset` part.**

In [ ]:
# select some motion sets for a particual figure
selector = {
    '_id':{'$gt': 0},
    'payload.d.motionset': {
        '$gte': '2018-12-16T08:00',  # $gte: greater than or equal
        '$lt': '2018-12-17T00:00' # $lte: less than or equal
    },
    'payload.d.figure': '4',
}

# Create the query ...
figure_query = Query(database, selector=selector, fields=fields)

# ... and get all motion sets
motionsets = sorted(set(e['payload']['d']['motionset'] for e in figure_query()['docs']))

# now pick one motionset and build a new query for the particular motionset
selector['payload.d.motionset'] = motionsets[1]
figure_query = Query(database, selector=selector, fields=fields)

In [ ]:
# make a sorted array of events and remove the payload.d prefix
values = sorted([e['payload']['d'] for e in figure_query()['docs']], key=lambda e: e['timestamp'])
values[:2]

In [ ]:
payload = {
    'values': values
}

In [ ]:
# now do the digit classification
digit_classification()(payload)

# Store and deploy the function

Before you can deploy the function, you must store the function in your Watson Machine Learning repository.

In [ ]:
#
# Store the deployable function in your Watson Machine Learning repository
#
meta_props = {
    wml_client.repository.FunctionMetaNames.NAME : 'Deployable Digit Classification'
}
function_details = wml_client.repository.store_function(meta_props=meta_props, function=digit_classification)

In [ ]:
#
# Deploy the stored function
#
artifact_uid = function_details["metadata"]["guid"]
function_deployment_details =wml_client.deployments.create(artifact_uid=artifact_uid, name="Digit Classification with preprocessing")

# Test the deployed function

You can use 

 1. the Watson Machine Learning Python client or 
 
 2. REST API

to send data to your function deployment for processing in exactly the same way you send data to model deployments for processing.

In [ ]:
# Get the endpoint URL of the function deployment just created
function_deployment_endpoint_url = wml_client.deployments.get_scoring_url( function_deployment_details )
function_deployment_endpoint_url

#### 1.) Watson Machine Learning Python client

In [ ]:
result = wml_client.deployments.score( function_deployment_endpoint_url, payload )
print( result )

### 2. Watson Machine Learning REST API

This is actually the way which is then used at the Raspi using two HTTP request nodes.

1. You get an access token from your WML instance using username and password
2. Use this access toke in the next request where the scoring payload is sent to the function endpoint (function_deployment_endpoint_url)

In [ ]:
import requests

# Get a bearer token
url = wml_credentials["url"] + "/v3/identity/token"
response = requests.get( url, auth=( wml_credentials["username"], wml_credentials["password"] ) )
mltoken = json.loads( response.text )["token"]

# Send sample canvas data to function deployment for processing
header = { 'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken }
response = requests.post( function_deployment_endpoint_url, json=payload, headers=header )
print ( response.text )

# Now its time to go to Raspi and ... 

... implement the call to the WEB service access to the function `digit_classification` in Node.RED.
Some remarks:

* In the HTTP request node "Get Token" you have to use the following url `wml_credentials["url"] + "/v3/identity/token"` (see result of cell below) and the user and password from wml_credentials
* In the HTTP request node "Classification Request" you have to set msg.url to the value of `function_deployment_endpoint_url` (see a few cells above)


In [ ]:
wml_credentials["url"] + "/v3/identity/token"